To install rascal:
(NOTE: See the top-level README for the most up-to-date installation instructions.)
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

In [ ]:
%matplotlib inline
from matplotlib import pylab as plt

import os, sys
from ase.io import read
sys.path.insert(0,"../build/")

import sys
import time
import rascal
import json

import ase
from ase.io import read, write
from ase.build import make_supercell
from ase.visualize import view
import numpy as np
import sys
import pandas as pd
from copy import deepcopy

from tqdm.notebook import tqdm

from rascal.representations import SphericalInvariants
from rascal.models import Kernel, KRR, train_gap_model, SparsePoints
from rascal.utils import from_dict, to_dict

In [ ]:
# import urllib.request
# # a collection of distorted ethanol molecules from the ANI-1 dataset 
# # (see https://github.com/isayev/ANI1_dataset) with energies and forces computed using DFTB+ 
# # (see https://www.dftbplus.org/)
# url = 'https://raw.githubusercontent.com/cosmo-epfl/librascal-example-data/833b4336a7daf471e16993158322b3ea807b9d3f/inputs/molecule_conformers_dftb.xyz'
# # Download the file from `url`, save it in a temporary directory and get the
# # path to it (e.g. '/tmp/tmpb48zma.txt') in the `structures_fn` variable:
# structures_fn, headers = urllib.request.urlretrieve(url)
# structures_fn
structures_fn = '/tmp/tmpiilc84vk'

# Spherical Invariants: body order = 3

## utils

In [ ]:
def extract_ref(frames,info_key='dft_formation_energy_per_atom_in_eV',array_key='zeros'):
    y,f = [], []
    for frame in frames:
        y.append(frame.info[info_key])
        if array_key is None:
            pass
        elif array_key == 'zeros':
            f.append(np.zeros(frame.get_positions().shape))
        else:
            f.append(frame.get_array(array_key))
    y= np.array(y)
    try:
        f = np.concatenate(f)
    except:
        pass
    return y,f

from scipy.stats import spearmanr

def get_r2(y_pred,y_true):
    weight = 1
    sample_weight = None
    numerator = (weight * (y_true - y_pred) ** 2).sum(axis=0,dtype=np.float64)
    denominator = (weight * (y_true - np.average(
        y_true, axis=0, weights=sample_weight)) ** 2).sum(axis=0,dtype=np.float64)
    output_scores = 1 - (numerator / denominator)
    return np.mean(output_scores)

def get_mae(ypred,y):
    return np.mean(np.abs(ypred-y))
def get_rmse(ypred,y):
    return np.sqrt(np.mean((ypred-y)**2))
def get_sup(ypred,y):
    return np.amax(np.abs((ypred-y)))
def get_spearman(ypred,y):
    corr,_ = spearmanr(ypred,y)
    return corr

score_func = dict(
    MAE=get_mae,
    RMSE=get_rmse,
    SUP=get_sup,
    R2=get_r2,
    CORR=get_spearman
)

def get_score(ypred,y):
    scores = {}
    for k,func in score_func.items():
        scores[k] = func(ypred,y)
    return scores
def print_score(ypred,y):
    scores = get_score(ypred,y)
    print(' '.join(map(lambda x:'{}={:.2e}'.format(*x), scores.items())))


## compute the representation of some atomic structures and their similarity

In [ ]:
# Load the small molecules 
frames = read('../reference_data/inputs/small_molecules-1000.xyz',':100')

In [ ]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.5, 
              max_radial=1, 
              max_angular=1, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=True,
              )
soap = SphericalInvariants(**hypers)
zeta=1
kernel1 = Kernel(soap,name='Cosine', zeta=zeta, target_type='Structure', kernel_type='Full')

In [ ]:
frames[0].info

In [ ]:
representation = soap.transform(frames)
X = representation.get_features(soap)

In [ ]:
%%time 
kk = np.power(np.dot(X, X.T), zeta)

## identify the most important features for regression with standard KRR

In [ ]:
def train_krr_model(kernel, managers, K_, y_train, sigma=1e-3, jitter=1e-8):
    Y = y_train.reshape((-1, 1)).copy()
    K = K_.copy()
    n_centers = Y.shape[0]
    Natoms = np.zeros(n_centers)
    Y0 = np.zeros((n_centers, 1))
    for iframe, manager in enumerate(managers):
        Natoms[iframe] = len(manager)
    delta = np.std(Y)
    # K[np.diag_indices_from(K)] *= (sigma / delta)**2 * Natoms + jitter
    K[np.diag_indices_from(K)] *= (sigma / delta)**2 + jitter
    weights = np.linalg.lstsq(K, Y, rcond=None)[0]
    model = KRR(weights, kernel, managers, {sp:0. for sp in range(120)})

    # avoid memory clogging
    del K
    K = []

    return model

In [ ]:
# Total number of structure to load
N = 985
# Number of structure to train the model with
f = int(0.6*N)

# load the structures
frames = read('../reference_data/inputs/small_molecules-1000.xyz',':{}'.format(N))


global_species = []
for frame in frames:
    global_species.extend(frame.get_atomic_numbers())
global_species = np.unique(global_species)

# split the structures in 2 sets
ids = list(range(N))
np.random.seed(10)
np.random.shuffle(ids)

frames_train = [frames[ii] for ii in ids[:f]]
frames_test = [frames[ii] for ii in ids[f:]]


In [ ]:
# split the dataset in a train and a test set
y_train, _ = extract_ref(frames_train,'dft_formation_energy_per_atom_in_eV')
for ii,ft in enumerate(frames_train):
    y_train[ii] /= len(ft)
y_test, _ = extract_ref(frames_test,'dft_formation_energy_per_atom_in_eV')
for ii,ft in enumerate(frames_test):
    y_test[ii] /= len(ft)

In [ ]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.5, 
              max_radial=8, 
              max_angular=8, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=True,
              expansion_by_species_method='structure wise',
              )
soap = SphericalInvariants(**hypers)
kernel = Kernel(soap,name='Cosine', zeta=2, target_type='Structure', kernel_type='Full')

In [ ]:
managers_train = soap.transform(frames_train)
managers_test = soap.transform(frames_test)
%time K = kernel(managers_train, managers_train)

In [ ]:
# train a KRR model
model = train_krr_model(kernel, managers_train, K, y_train, sigma=0.5e-1, jitter=1e-8)

# make predictions on the test set
y_pred = model.predict(managers_test)

# basic assessement of the quality of the trained model
print_score(y_pred, y_test)
plt.plot(y_test, y_pred, 'o')
plt.title("correlation plot")
plt.ylabel("predicted energies [eV]")
plt.xlabel("reference energies [eV]")

## Sparsification with the gap model

In [ ]:
from rascal.utils import BaseIO
from rascal.models import SparsePoints

from rascal.lib import sparsification
from rascal.utils.filter_utils import (get_index_mappings_sample_per_species,
                           convert_selected_global_index2rascal_sample_per_species,
                           get_index_mappings_sample,
                           convert_selected_global_index2rascal_sample)

import numpy as np
from scipy.sparse.linalg import svds


def do_CUR(X, Nsel, act_on='sample', is_deterministic=False, seed=10, verbose=True):
    """ Apply CUR selection [1] of Nsel rows or columns of the
    given feature matrix X[n_samples, n_features].

    .. [1] Mahoney, M. W., & Drineas, P. (2009). CUR matrix decompositions for
    improved data analysis. Proceedings of the National Academy of Sciences,106(3),
    697–702. https://doi.org/10.1073/pnas.0803205106
    """
    U, _, VT = svds(X, Nsel)
    if 'sample' in act_on:
        weights = np.mean(np.square(U), axis=1)
    elif 'feature' in act_on:
        weights = np.mean(np.square(VT), axis=0)
    if is_deterministic is True:
        # sorting is smallest to largest hence the minus
        sel = np.argsort(-weights)[:Nsel]
    elif is_deterministic is False:
        np.random.seed(seed)
        # sorting is smallest to largest hence the minus
        sel = np.argsort(np.random.rand(*weights.shape) - weights)[:Nsel]

    if verbose is True:
        if 'sample' in act_on:
            C = X[sel, :]
            Cp = np.linalg.pinv(C)
            err = np.sqrt(np.sum((X - np.dot(np.dot(X, Cp), C))**2))
        elif 'feature' in act_on:
            C = X[:, sel]
            Cp = np.linalg.pinv(C)
            err = np.sqrt(np.sum((X - np.dot(C, np.dot(Cp, X)))**2))

        print('Reconstruction RMSE={:.3e}'.format(err))

    return sel


class CURFilter(BaseIO):
    """CUR decomposition to select samples or features in a given feature matrix.
    Wrapper around the do_CUR function for convenience.

    Parameters
    ----------
    representation : Calculator
        Representation calculator associated with the kernel

    Nselect: int
        number of points to select. if act_on='sample per species' then it should
        be a dictionary mapping atom type to the number of samples, e.g.
        Nselect = {1:200,6:100,8:50}.

    act_on: string
        Select how to apply the selection. Can be either of 'sample',
        'sample per species','feature'.
        For the moment only 'sample per species' is implemented.

    is_deterministic: bool
        flag to switch between selction criteria

    seed: int
        if is_deterministic==False, seed for the random selection

    """

    def __init__(self, representation, Nselect, act_on='sample per species',
                                                is_deterministic=True, seed=10):
        self._representation = representation
        self.Nselect = Nselect
        if act_on in ['sample', 'sample per species', 'feature']:
            self.act_on = act_on
        else:
            raise ValueError(
                '"act_on" should be either of: "{}", "{}", "{}"'.format(
                    *['sample', 'sample per species', 'feature']))
        self.is_deterministic = is_deterministic
        self.seed = seed
        self.selected_ids = None
        self.selected_ids_by_sp = None
        self.selected_ids_global = None

        
    def fit(self, managers):
        """Perform CUR selection of samples/features.

        Parameters
        ----------
        managers : AtomsList
            list of structures containing features computed with representation

        Returns
        -------
        SparsePoints
            Selected samples

        Raises
        ------
        ValueError
            [description]
        NotImplementedError
            [description]
        """
        # get the dense feature matrix
        X = managers.get_features(self._representation)

        if self.act_on in ['sample per species']:
            sps = list(self.Nselect.keys())

            # get various info from the structures about the center atom species and indexing
            (strides_by_sp, global_counter, map_by_manager,
             indices_by_sp) = get_index_mappings_sample_per_species(managers, sps)

            print('The number of pseudo points selected by central atom species is: {}'.format(
                self.Nselect))

            # organize features w.r.t. central atom type
            X_by_sp = {}
            for sp in sps:
                X_by_sp[sp] = X[indices_by_sp[sp]]
            self._XX = X_by_sp

            # split the dense feature matrix by center species and apply CUR decomposition
            self.selected_ids_by_sp = {}
            self.fps_minmax_d2_by_sp = {}
            self.fps_hausforff_d2_by_sp = {}
            for sp in sps:
                print('Selecting species: {}'.format(sp))
                self.selected_ids_by_sp[sp] = do_CUR(X_by_sp[sp], self.Nselect[sp], self.act_on,
                                                        self.is_deterministic, self.seed)

        elif self.act_on in ['sample']:
            self.selected_ids = do_CUR(X, self.Nselect, self.act_on,
                                               self.is_deterministic, self.seed)
        elif self.act_on in ['feature']:                                                
            self.selected_ids = do_CUR(X, self.Nselect, self.act_on,
                                                        self.is_deterministic, self.seed)
        else:
            raise ValueError("method: {}".format(self.act_on))
                    
        return self
    
    def transform(self, managers, n_select=None):
        if n_select is None:
            n_select = self.Nselect
            
        if self.act_on in ['sample per species']:
            sps = list(n_select.keys())
            # get various info from the structures about the center atom species and indexing
            (strides_by_sp, global_counter, map_by_manager,
             indices_by_sp) = get_index_mappings_sample_per_species(managers, sps)
            selected_ids_by_sp = {key: np.sort(val[:n_select[key]])
                                  for key, val in self.selected_ids_by_sp.items()}
            self.selected_ids = convert_selected_global_index2rascal_sample_per_species(
                managers, selected_ids_by_sp, strides_by_sp, map_by_manager, sps)
            # return self.selected_ids
            # build the pseudo points
            pseudo_points = SparsePoints(self._representation)
            pseudo_points.extend(managers, self.selected_ids)
            return pseudo_points
        
        elif self.act_on in ['sample']:
            selected_ids_global = np.sort(self.selected_ids_global[:n_select])
            strides, _, map_by_manager = get_index_mappings_sample(managers)
            self.selected_ids = convert_selected_global_index2rascal_sample(managers,
                                                selected_ids_global, strides, map_by_manager)
            return self.selected_ids
            # # build the pseudo points
            # pseudo_points = SparsePoints(self._representation)
            # pseudo_points.extend(managers, self.selected_ids)
            # return pseudo_points

        elif self.act_on in ['feature']:
            feat_idx2coeff_idx = self._representation.get_feature_index_mapping(
                managers)
            selected_features = {key: []
                                 for key in feat_idx2coeff_idx[0].keys()}
            selected_ids_sorting = np.argsort(self.selected_ids[:n_select])
            selected_ids = self.selected_ids[selected_ids_sorting]
            for idx in selected_ids:
                coef_idx = feat_idx2coeff_idx[idx]
                for key in selected_features.keys():
                    selected_features[key].append(int(coef_idx[key]))
            # keep the global indices and ordering for ease of use
            selected_features['selected_features_global_ids'] = selected_ids.tolist()
            selected_features['selected_features_global_ids_fps_ordering'] = selected_ids_sorting.tolist()
            return dict(coefficient_subselection=selected_features)
            
    def fit_transform(self, managers):
        return self.fit(managers).transform(managers)
    def _get_data(self):
        return dict(selected_ids=self.selected_ids, 
                    selected_ids_by_sp=self.selected_ids_by_sp,
                   selected_ids_global=self.selected_ids_global)

    def _set_data(self, data):
        self.selected_ids = data['selected_ids']
        self.selected_ids_by_sp = data['selected_ids_by_sp']
        self.selected_ids_global = data['selected_ids_global']
    def _get_init_params(self):
        return dict(representation=self._representation,
                    Nselect=self.Nselect,
                    act_on=self.act_on,
                    is_deterministic=self.is_deterministic,
                    seed=self.seed,)

In [ ]:

def fps(feature_matrix, n_select, starting_index=None,
        method='simple', restart=None):
    """
    Farthest Point Sampling [1] routine using librascal.

    Parameters
    ----------
    feature_matrix : numpy.ndarray[float64[n, m], flags.c_contiguous]
        Feature matrix with n samples and m features.
    n_select : int
        Number of sample to select
    starting_index : int, optional
        Index of the first sample to select (the default is None,
                           which corresponds to starting_index == 0)
    restart : dictionary, optional (only valid for method="simple")
        the return value of a previous call to FPS. the selection
        will continue where it has been left off. the dictionary
        should contain at least "fps_indices"; if "fps_minmax_d2" and
        "fps_hausdorff_d2" are present, they will be used to restart,
        otherwise they'll be recomputed

    method : str, optional
        Select which kind of FPS selection to perform:
        + 'simple' is the basic fps selection [1].
        + 'voronoi' uses voronoi polyhedra to avoid some
                    distance computations.

    Returns
    -------
    A dictionary containing the following quantities
    "fps_indices" : numpy.ndarray[int[n_select]]
        Selected indices refering to the feature_matrix order.
    "fps_minmax_d2": numpy.ndarray[float64[n_select]]
        MIN-MAX distance^2 at each step in the selection
    "fps_hausforff_d2": numpy.ndarray[float64[n]]
        array of Hausdorff distances between the n points and the
        n_select FPS points
    In addition, when method="voronoi", the following arrays are returned
    "fps_voronoi_indices": numpy.ndarray[int[n]]
        the indices that assign each of the n inputs to the closest FPS point
    "fps_voronoi_r2": numpy.ndarray[float64[n_select]]
        the squared "Voronoi radius" of each FPS point (the largest distance to
        one of the points associated to its Voronoi cell


    .. [1] Ceriotti, M., Tribello, G. A., & Parrinello, M. (2013).
        Demonstrating the Transferability and the Descriptive Power of Sketch-Map.
        Journal of Chemical Theory and Computation, 9(3), 1521–1532.
        https://doi.org/10.1021/ct3010563
    """

    if starting_index is None:
        starting_index = 0

    return_dict = {}
    if method == 'simple':
        if restart is None:
            sparse_indices, sparse_minmax_d2, lmin_d2 = \
                sparsification.fps(feature_matrix, n_select, starting_index)
        else:
            res_tuple = (restart["fps_indices"],
                         (restart["fps_minmax_d2"] if "fps_minmax_d2" in restart
                          else np.zeros(0, float)),
                         (restart["fps_hausdorff_d2"] if "fps_hausdorff_d2" in restart
                          else np.zeros(0, float)))
            sparse_indices, sparse_minmax_d2, lmin_d2 = \
                sparsification.fps(feature_matrix, n_select,
                                   starting_index, res_tuple)

    elif method == 'voronoi':
        sparse_indices, sparse_minmax_d2, lmin_d2, \
            voronoi_indices, voronoi_r2 = \
            sparsification.fps_voronoi(feature_matrix,
                                       n_select, starting_index)
        return_dict["fps_voronoi_indices"] = voronoi_indices
        return_dict["fps_voronoi_r2"] = voronoi_r2

    else:
        raise Exception('Unknown FPS algorithm {}'.format(method))

    return_dict["fps_indices"] = sparse_indices
    return_dict["fps_minmax_d2"] = sparse_minmax_d2
    return_dict["fps_hausdorff_d2"] = lmin_d2

    return return_dict


class FPSFilter(BaseIO):
    """Farther Point Sampling (FPS) to select samples or features in a given feature matrix.
    Wrapper around the fps function for convenience.
    Parameters
    ----------
    representation : Calculator
        Representation calculator associated with the kernel
    Nselect: int
        number of points to select. if act_on='sample per specie' then it should
        be a dictionary mapping atom type to the number of samples, e.g.
        Nselect = {1:200,6:100,8:50}.
    act_on: string
        Select how to apply the selection. Can be either of 'sample',
        'sample per species','feature'.
    starting_index: int
        Index used to start the FPS selection with.

    """

    def __init__(self, representation, Nselect, act_on='sample per species', starting_index=0):
        self._representation = representation
        self.Nselect = Nselect
        self.starting_index = starting_index
        if act_on in ['sample', 'sample per species', 'feature']:
            self.act_on = act_on
        else:
            raise 'Wrong input: {}'.format(act_on)

        self.selected_ids = None
        self.selected_ids_by_sp = None
        self.selected_ids_global = None
        self.fps_minmax_d2_by_sp = None
        self.fps_minmax_d2 = None

    def fit(self, managers):
        """Perform FPS selection of samples/features.
        Parameters
        ----------
        managers : AtomsList
            list of structures containing features computed with representation
        Returns
        -------
        PseudoPoints
            Selected samples
        Raises
        ------
        ValueError
            [description]
        NotImplementedError
            [description]
        """

        # get the dense feature matrix
        X = managers.get_features(self._representation)

        if self.act_on in ['sample per species']:
            sps = list(self.Nselect.keys())

            # get various info from the structures about the center atom species and indexing
            (strides_by_sp, global_counter, map_by_manager,
             indices_by_sp) = get_index_mappings_sample_per_species(managers, sps)

            print('The number of pseudo points selected by central atom species is: {}'.format(
                self.Nselect))

            # organize features w.r.t. central atom type
            X_by_sp = {}
            for sp in sps:
                X_by_sp[sp] = X[indices_by_sp[sp]]
            self._XX = X_by_sp

            # split the dense feature matrix by center species and apply CUR decomposition
            self.selected_ids_by_sp = {}
            self.fps_minmax_d2_by_sp = {}
            self.fps_hausforff_d2_by_sp = {}
            for sp in sps:
                print('Selecting species: {}'.format(sp))
                fps_out = fps(X_by_sp[sp], self.Nselect[sp],
                              starting_index=self.starting_index)
                self.selected_ids_by_sp[sp] = fps_out['fps_indices']
                self.fps_minmax_d2_by_sp[sp] = fps_out['fps_minmax_d2']

            return self
        elif self.act_on in ['feature']:
            fps_out = fps(X.T, self.Nselect, starting_index=self.starting_index)
            self.selected_ids = fps_out['fps_indices']
            self.fps_minmax_d2 = fps_out['fps_minmax_d2']
        elif self.act_on in ['sample']:
            fps_out = fps(X, self.Nselect, starting_index=self.starting_index)
            self.selected_ids_global = fps_out['fps_indices']
            self.fps_minmax_d2 = fps_out['fps_minmax_d2']
        else:
            raise NotImplementedError("method: {}".format(self.act_on))

    def transform(self, managers, n_select=None):
        if n_select is None:
            n_select = self.Nselect
            
        if self.act_on in ['sample per species']:
            sps = list(n_select.keys())
            # get various info from the structures about the center atom species and indexing
            (strides_by_sp, global_counter, map_by_manager,
             indices_by_sp) = get_index_mappings_sample_per_species(managers, sps)
            selected_ids_by_sp = {key: np.sort(val[:n_select[key]])
                                  for key, val in self.selected_ids_by_sp.items()}
            self.selected_ids = convert_selected_global_index2rascal_sample_per_species(
                managers, selected_ids_by_sp, strides_by_sp, map_by_manager, sps)
            # return self.selected_ids
            # build the pseudo points
            pseudo_points = SparsePoints(self._representation)
            pseudo_points.extend(managers, self.selected_ids)
            return pseudo_points
        
        elif self.act_on in ['sample']:
            selected_ids_global = np.sort(self.selected_ids_global[:n_select])
            strides, _, map_by_manager = get_index_mappings_sample(managers)
            self.selected_ids = convert_selected_global_index2rascal_sample(managers,
                                                                            selected_ids_global, strides, map_by_manager)
            return self.selected_ids
            # # build the pseudo points
            # pseudo_points = SparsePoints(self._representation)
            # pseudo_points.extend(managers, self.selected_ids)
            # return pseudo_points

        elif self.act_on in ['feature']:
            feat_idx2coeff_idx = self._representation.get_feature_index_mapping(
                managers)
            selected_features = {key: []
                                 for key in feat_idx2coeff_idx[0].keys()}
            selected_ids_sorting = np.argsort(self.selected_ids[:n_select])
            selected_ids = self.selected_ids[selected_ids_sorting]
            for idx in selected_ids:
                coef_idx = feat_idx2coeff_idx[idx]
                for key in selected_features.keys():
                    selected_features[key].append(int(coef_idx[key]))
            # keep the global indices and ordering for ease of use
            selected_features['selected_features_global_ids'] = selected_ids.tolist()
            selected_features['selected_features_global_ids_fps_ordering'] = selected_ids_sorting.tolist()
            return dict(coefficient_subselection=selected_features)
    
    def fit_transform(self, managers):
        return self.fit(managers).transform(managers)
    
    def plot(self):
        import matplotlib.pyplot as plt
        if self.fps_minmax_d2_by_sp is None:
            plt.semilogy(self.fps_minmax_d2, label=self.act_on)

        else:
            for sp in self.fps_minmax_d2_by_sp:
                plt.semilogy(self.fps_minmax_d2_by_sp[sp],
                             label='{} species {}'.format(self.act_on, sp))
            plt.legend()
        plt.title('FPSFilter')
        plt.ylabel('fps minmax d^2')

    def fit_transform(self, managers):
        return self.fit(managers).transform(managers)
    
    def _get_data(self):
        return dict(selected_ids=self.selected_ids, 
                    selected_ids_by_sp=self.selected_ids_by_sp,
                   selected_ids_global=self.selected_ids_global)

    def _set_data(self, data):
        self.selected_ids = data['selected_ids']
        self.selected_ids_by_sp = data['selected_ids_by_sp']
        self.selected_ids_global = data['selected_ids_global']
    def _get_init_params(self):
        return dict(representation=self._representation,
                    Nselect=self.Nselect,
                    act_on=self.act_on,
                    starting_index=self.starting_index,)

In [ ]:
# Total number of structure to load
N = 1000
# Number of structure to train the model with
f = int(0.6*N)

# load the structures
frames = read(structures_fn,':{}'.format(N))


global_species = []
for frame in frames:
    global_species.extend(frame.get_atomic_numbers())
global_species = np.unique(global_species)

# split the structures in 2 sets
ids = list(range(N))
np.random.seed(10)
np.random.shuffle(ids)

frames_train = [frames[ii] for ii in ids[:f]]
frames_test = [frames[ii] for ii in ids[f:]]
# Isolated atom contributions
self_contributions = {
    1: -6.492647589968434,
    6: -38.054950840332474,
    8: -83.97955098636527,
}

In [ ]:
# split the dataset in a train and a test set
y_train, _ = extract_ref(frames_train,'dftb_energy_eV')
y_test, _ = extract_ref(frames_test,'dftb_energy_eV')

In [ ]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.5, 
              max_radial=8, 
              max_angular=8, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              normalize=True,
              expansion_by_species_method='structure wise',
              )
soap_calculator = SphericalInvariants(**hypers)
kernel = Kernel(soap_calculator,name='GAP', zeta=2, target_type='Structure', kernel_type='Sparse')

### pick up randomly sparse points

In [ ]:
Nenv = 0
env_map = []
for i_frame, frame in enumerate(frames):
    Nenv += len(frame)
    for i_env in range(len(frame)):
        env_map.append((i_frame, i_env))
Nenv

In [ ]:
managers = soap_calculator.transform(frames)
managers_train = soap_calculator.transform(frames_train)
managers_test = soap_calculator.transform(frames_test)

In [ ]:
# compute the representation for all atomic structures
managers = soap_calculator.transform(frames)

fractions = (Nenv*np.array([0.4,0.3,0.2,0.1,0.05])).astype(int)
scores = []
for fraction in tqdm(fractions):
    ids = np.array(range(Nenv))
    np.random.seed(100)
    np.random.shuffle(ids)
    # randomly select atomic centers
    selected_ids = [[] for _ in range(len(frames))]
    for idx in ids[:fraction]:
        i_frame, i_env = env_map[idx]
        selected_ids[i_frame].append(i_env)
    # initialize the sparse points with randomly selected 
    sparse_points = SparsePoints(soap_calculator)
    sparse_points.extend(managers, selected_ids)
    
    KNM = kernel(managers_train, sparse_points)
    
    model = train_gap_model(kernel, managers_train, KNM, sparse_points, y_train, self_contributions, 
                        grad_train=None, lambdas=[7e-3, None], jitter=1e-8)

    y_pred = model.predict(managers_test)
    
    score = get_score(y_pred, y_test)
    score.update(n_sparse_point=fraction)
    scores.append(score)
scores = pd.DataFrame(scores)
scores

### pick sparse points with fps

In [ ]:
managers = soap_calculator.transform(frames)
managers_train = soap_calculator.transform(frames_train)
managers_test = soap_calculator.transform(frames_test)

In [ ]:
sps = []
for frame in frames:
    sps.extend(frame.get_atomic_numbers())
sps_u = np.unique(sps)
sps_n = np.bincount(sps)
Nenv = {sp:sps_n[sp] for sp in sps_u}
Nenv

In [ ]:
fps_filter = FPSFilter(soap_calculator, Nenv, 'sample per species')
fps_filter.fit(managers)
fps_filter.plot()

In [ ]:
managers = soap_calculator.transform(frames)

fractions = [0.4,0.3,0.2,0.1,0.05]
scores = []
for fraction in tqdm(fractions):
    sparse_points = fps_filter.transform(managers, {sp:int(n*fraction) for sp,n in Nenv.items()})
    KNM = kernel(managers_train, sparse_points)
    model = train_gap_model(kernel, managers_train, KNM, sparse_points, y_train, self_contributions, 
                        grad_train=None, lambdas=[7e-3, None], jitter=1e-8)

    y_pred = model.predict(managers_test)
    score = get_score(y_pred, y_test)
    score.update(fraction=fraction)
    scores.append(score)
scores = pd.DataFrame(scores)
scores

In [ ]:
fraction = 0.2
sparse_points = fps_filter.transform(managers, {sp:int(n*fraction) for sp,n in Nenv.items()})
KNM = kernel(managers_train, sparse_points)

In [ ]:
# train a GAP model 
model = train_gap_model(kernel, managers_train, KNM, sparse_points, y_train, self_contributions, 
                        grad_train=None, lambdas=[7e-3, None], jitter=1e-8)

y_pred = model.predict(managers_test)

print_score(y_pred, y_test)

plt.plot(y_test, y_pred, 'o')
plt.title("correlation plot")
plt.xlabel("predicted energies [eV]")
plt.ylabel("reference energies [eV]")

### try feature sparsification with FPS

In [ ]:
managers = soap_calculator.transform(frames)

X = managers.get_features(soap_calculator)
n_features = int(X.shape[1]*0.4)
n_features

In [ ]:
sps = []
for frame in frames:
    sps.extend(frame.get_atomic_numbers())
sps_u = np.unique(sps)
sps_n = np.bincount(sps)
Nenv = {sp:int(sps_n[sp]*0.05) for sp in sps_u}
print(Nenv)
fps_filter = FPSFilter(soap_calculator, Nenv, 'sample per species')
fps_filter.fit(managers)
fps_filter.plot()

In [ ]:
feature_filter = FPSFilter(soap_calculator, n_features, 'feature')
feature_filter.fit(managers);
feature_filter.plot()

In [ ]:
fractions = list(reversed([1, 0.75, 0.5, 0.25, 0.125, 0.05, 0.01]))
scores = []
for fraction in tqdm(fractions):
    selected_feature = feature_filter.transform(managers, int(n_features*fraction))
    
    hypers_sparse = deepcopy(hypers)
    hypers_sparse.update(**selected_feature)
    soap_calculator_sparse = SphericalInvariants(**hypers_sparse)
    
    kernel_sparse = Kernel(soap_calculator_sparse,name='GAP', zeta=2, target_type='Structure', kernel_type='Sparse')
    
    managers_sp = soap_calculator_sparse.transform(frames)
    managers_train_sp = soap_calculator_sparse.transform(frames_train)
    managers_test_sp = soap_calculator_sparse.transform(frames_test)
    
    fps_filter_sp = from_dict(to_dict(fps_filter))
    fps_filter_sp._representation = soap_calculator_sparse
    sparse_points = fps_filter_sp.transform(managers_sp)
    
    KNM = kernel_sparse(managers_train_sp, sparse_points)
    model = train_gap_model(kernel_sparse, managers_train_sp, KNM, sparse_points, y_train, self_contributions, 
                        grad_train=None, lambdas=[7e-3, None], jitter=1e-8)

    y_pred = model.predict(managers_test_sp)
    score = get_score(y_pred, y_test)
    score.update(n_features=int(n_features*fraction))
    scores.append(score)
scores = pd.DataFrame(scores)
scores

### pick sparse points with CUR decomposition

In [ ]:
managers = soap_calculator.transform(frames)
managers_train = soap_calculator.transform(frames_train)
managers_test = soap_calculator.transform(frames_test)

In [ ]:
sps = []
for frame in frames:
    sps.extend(frame.get_atomic_numbers())
sps_u = np.unique(sps)
sps_n = np.bincount(sps)
Nenv = {sp:int(sps_n[sp]*0.4) for sp in sps_u}
Nenv

In [ ]:
cur_filter = CURFilter(soap_calculator, Nenv, 'sample per species')
cur_filter.fit(managers)

In [ ]:
managers = soap_calculator.transform(frames)

fractions = [1, 0.75, 0.5, 0.25, 0.125]
scores = []
for fraction in tqdm(fractions):
    sparse_points = cur_filter.transform(managers, {sp:int(n*fraction) for sp,n in Nenv.items()})
    KNM = kernel(managers_train, sparse_points)
    model = train_gap_model(kernel, managers_train, KNM, sparse_points, y_train, self_contributions, 
                        grad_train=None, lambdas=[7e-3, None], jitter=1e-8)

    y_pred = model.predict(managers_test)
    score = get_score(y_pred, y_test)
    score.update(fraction=fraction)
    scores.append(score)
scores = pd.DataFrame(scores)
scores

In [ ]:
fraction = 0.2
sparse_points = fps_filter.transform(managers, {sp:int(n*fraction) for sp,n in Nenv.items()})
KNM = kernel(managers_train, sparse_points)

In [ ]:
# train a GAP model 
model = train_gap_model(kernel, managers_train, KNM, sparse_points, y_train, self_contributions, 
                        grad_train=None, lambdas=[7e-3, None], jitter=1e-8)

y_pred = model.predict(managers_test)

print_score(y_pred, y_test)

plt.plot(y_test, y_pred, 'o')
plt.title("correlation plot")
plt.xlabel("predicted energies [eV]")
plt.ylabel("reference energies [eV]")

### try feature sparsification with CUR

In [ ]:
managers = soap_calculator.transform(frames)

X = managers.get_features(soap_calculator)
n_features = int(X.shape[1]*0.4)
n_features

In [ ]:
sps = []
for frame in frames:
    sps.extend(frame.get_atomic_numbers())
sps_u = np.unique(sps)
sps_n = np.bincount(sps)
Nenv = {sp:int(sps_n[sp]*0.05) for sp in sps_u}
print(Nenv)
cur_filter = CURFilter(soap_calculator, Nenv, 'sample per species')
cur_filter.fit(managers);

In [ ]:
feature_filter = CURFilter(soap_calculator, n_features, 'feature')
feature_filter.fit(managers);

In [ ]:
fractions = list(reversed([1, 0.75, 0.5, 0.25, 0.125, 0.05, 0.01]))
scores = []
for fraction in tqdm(fractions):
    selected_feature = feature_filter.transform(managers, int(n_features*fraction))
    
    hypers_sparse = deepcopy(hypers)
    hypers_sparse.update(**selected_feature)
    soap_calculator_sparse = SphericalInvariants(**hypers_sparse)
    
    kernel_sparse = Kernel(soap_calculator_sparse,name='GAP', zeta=2, target_type='Structure', kernel_type='Sparse')
    
    managers_sp = soap_calculator_sparse.transform(frames)
    managers_train_sp = soap_calculator_sparse.transform(frames_train)
    managers_test_sp = soap_calculator_sparse.transform(frames_test)
    
    cur_filter_sp = from_dict(to_dict(cur_filter))
    cur_filter_sp._representation = soap_calculator_sparse
    sparse_points = cur_filter_sp.transform(managers_sp)
    
    KNM = kernel_sparse(managers_train_sp, sparse_points)
    model = train_gap_model(kernel_sparse, managers_train_sp, KNM, sparse_points, y_train, self_contributions, 
                        grad_train=None, lambdas=[7e-3, None], jitter=1e-8)

    y_pred = model.predict(managers_test_sp)
    score = get_score(y_pred, y_test)
    score.update(n_features=int(n_features*fraction))
    scores.append(score)
scores = pd.DataFrame(scores)
scores

# Make a map of the dataset

## utils

In [ ]:
def link_ngl_wdgt_to_ax_pos(ax, pos, ngl_widget):
    from matplotlib.widgets import AxesWidget
    from scipy.spatial import cKDTree
    r"""
    Initial idea for this function comes from @arose, the rest is @gph82 and @clonker
    """
    
    kdtree = cKDTree(pos)        
    #assert ngl_widget.trajectory_0.n_frames == pos.shape[0]
    x, y = pos.T
    
    lineh = ax.axhline(ax.get_ybound()[0], c="black", ls='--')
    linev = ax.axvline(ax.get_xbound()[0], c="black", ls='--')
    dot, = ax.plot(pos[0,0],pos[0,1], 'o', c='red', ms=7)

    ngl_widget.isClick = False
    
    def onclick(event):
        linev.set_xdata((event.xdata, event.xdata))
        lineh.set_ydata((event.ydata, event.ydata))
        data = [event.xdata, event.ydata]
        _, index = kdtree.query(x=data, k=1)
        dot.set_xdata((x[index]))
        dot.set_ydata((y[index]))
        ngl_widget.isClick = True
        ngl_widget.frame = index
    
    def my_observer(change):
        r"""Here comes the code that you want to execute
        """
        ngl_widget.isClick = False
        _idx = change["new"]
        try:
            dot.set_xdata((x[_idx]))
            dot.set_ydata((y[_idx]))            
        except IndexError as e:
            dot.set_xdata((x[0]))
            dot.set_ydata((y[0]))
            print("caught index error with index %s (new=%s, old=%s)" % (_idx, change["new"], change["old"]))
    
    # Connect axes to widget
    axes_widget = AxesWidget(ax)
    axes_widget.connect_event('button_release_event', onclick)
    
    # Connect widget to axes
    ngl_widget.observe(my_observer, "frame", "change")

## make a map with kernel pca projection

In [ ]:
# Load the small molecules 
frames = read('../reference_data/inputs/small_molecules-1000.xyz',':600')

In [ ]:
hypers = dict(soap_type="PowerSpectrum",
              interaction_cutoff=3.5, 
              max_radial=6, 
              max_angular=6, 
              gaussian_sigma_constant=0.4,
              gaussian_sigma_type="Constant",
              cutoff_smooth_width=0.5,
              )
soap = SphericalInvariants(**hypers)
kernel = Kernel(soap,name='Cosine', zeta=2, target_type='Structure', kernel_type='Full')

In [ ]:
managers = soap.transform(frames)

Kmat = kernel(managers)

In [ ]:
from sklearn.decomposition import KernelPCA

In [ ]:
kpca = KernelPCA(n_components=2,kernel='precomputed')
kpca.fit(Kmat)

In [ ]:
X = kpca.transform(Kmat)

In [ ]:
plt.scatter(X[:,0],X[:,1],s=3)

## make an interactive map

In [ ]:
# package to visualize the structures in the notebook
# https://github.com/arose/nglview#released-version
import nglview

In [ ]:
iwdg = nglview.show_asetraj(frames)
# set up the visualization
iwdg.add_unitcell()
iwdg.add_spacefill()
iwdg.remove_ball_and_stick()
iwdg.camera = 'orthographic'
iwdg.parameters = { "clipDist": 0 }
iwdg.center()
iwdg.update_spacefill(radiusType='covalent',
                                   scale=0.6,
                                   color_scheme='element')
iwdg._remote_call('setSize', target='Widget',
                               args=['%dpx' % (600,), '%dpx' % (400,)])
iwdg.player.delay = 200.0

In [ ]:
link_ngl_wdgt_to_ax_pos(plt.gca(), X, iwdg)
plt.scatter(X[:,0],X[:,1],s=3)
iwdg